Group Number : 30 
Group Name : SokoRamp

Group Members :
2010110121 - Appala Venkata Avinash
2010110193 - Bodduluri Saran
2010110340 - Katakam Saideep Reddy
2010110980 - Boyapati Hemanth Sai
2010111021 - Mohana Siddhartha Chivukula

In [1]:
!pip install scipy
!pip install numpy
!pip install fibheap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for fibheap: filename=fibheap-0.2.1-py3-none-any.whl size=5887 sha256=8a78f037ce9b0d4f912b728dab2472745d330354b219dd84bbba4d57898b5889
  Stored in directory: /root/.cache/pip/wheels/ee/c7/07/4b76aa333410f48bc886b4ec032f3a362e7e5b6b36002b3cc4
Successfully built fibheap


In [2]:
import math
import time
from copy import deepcopy
import queue
import heapq
from collections import deque
from queue import PriorityQueue
import sys
import numpy as np
from fibheap import *
from scipy.optimize import linear_sum_assignment

In [3]:
INFINITY = sys.maxsize

In [4]:
strucone = [[1]]

In [5]:
walls=set()
goals=set()
empty_tiles=set()
deadlocks=set()
dist_to_goal=dict()
move_directions=[(-1,0),(strucone[0][0],0),(0,-1),(0,strucone[0][0])]
class State:
    def __init__(self,move_history):
        self.move_history=move_history
        self.blocks=set()
        self.player=(0,0)
        self.g=0
        self.f=0
        self.block_on_goal=set()
        self.height=0
        self.width=0
        self.reqarr=[]
    def init_reqarr(self,array):
      self.reqarr=deepcopy(array)
      # print(self.reqarr)
    def goalpull(self):
        # print(self.goals)
        # print(self.empty_tiles)
        
        for i in goals:
            dist_to_goal[i]=dict()
            for j in empty_tiles.union(walls):
                if(i==j):
                    dist_to_goal[i][j]=0
                else:
                    dist_to_goal[i][j]=999999
        # print(self.dist_to_goal)
        for goal in dist_to_goal:
            # print(goal)
            goal_pull_q=deque()
            goal_pull_q.append(goal)
            while goal_pull_q:
                # print(goal_pull_q)
                position=goal_pull_q.popleft()
                for dir in move_directions:
                    block_pos=(position[0]+dir[0],position[1]+dir[1])
                    player_pos=(position[0]+2*dir[0],position[1]+2*dir[1])
                    # if block_pos[0] in range(1,self.height+1) and block_pos[1] in range(1,self.width+1) and player_pos[0] in range(1,self.height+1) and player_pos[1] in range(1,self.width+1):
                    if dist_to_goal[goal][block_pos]==999999:
                            # print(goal,block_pos)
                            if block_pos not in walls and player_pos not in walls:
                                dist_to_goal[goal][block_pos]=dist_to_goal[goal][position]+1
                                goal_pull_q.append(block_pos)
        # print(self.dist_to_goal)

    def __eq__(self,other):
        if(self.player==other.player and all(ind in other.blocks for ind in self.blocks)):
            return True
        else:
            return False
    def __hash__(self):
        boxhash=0
        for e in self.blocks:
            boxhash+=hash(e)
            boxhash=boxhash*37
        return self.player[0]*73+self.player[1]+boxhash

    def __gt__(self,other):
        if self.f>other.f:
            return True
        else:
            return False
    def __lt__(self,other):
        if self.f<other.f:
            return True
        else:
            return False
    def add_deadlocks(self,row_size,column_size):
        for i in range(1,row_size+1):
            for j in range(1,column_size+1):
                point=(i,j)
                if point not in walls and point not in goals:
                    if self.test_corner(point) or self.test_boundary(point,row_size,column_size):
                        deadlocks.add(point)
        
    def test_corner(self,point):
        top=(point[0]-1,point[1])
        down=(point[0]+1,point[1])
        left=(point[0],point[1]-1)
        right=(point[0],point[1]+1)
        
        if((top in walls and right in walls) or (top in walls and left in walls) 
        or (down in walls and right in walls) or (down in walls and left in walls)):
            return True
        return False
    def upbound_nearest(self,point,row_size,col_size):
        x=point[0]-1
        y=point[1]
        while x>=0:
            if (x,y) in walls:
                return x
            elif (x,y) in goals:
                return -1
            x=x-1
        return 0
    def downbound_nearest(self,point,row_size,col_size):
        x=point[0]+1
        y=point[1]
        while x<=row_size:
            if (x,y) in walls:
                return x
            elif (x,y) in goals:
                return -1
            x=x+1
        return row_size
    def rightbound_nearest(self,point,row_size,col_size):
        x=point[0]
        y=point[1]+1
        while y<col_size:
            if (x,y) in walls:
                return y
            elif(x,y) in goals:
                return -1
            y+=1
        return col_size
    def leftbound_nearest(self,point,row_size,col_size):
        x=point[0]
        y=point[1]-1
        while y>=0:
            if (x,y) in walls:
                return y
            elif (x,y) in goals:
                return -1
            y-=1
        return 0

    def test_boundary(self,point,row_size,col_size):
        top=(point[0]-1,point[1])
        down=(point[0]+1,point[1])
        left=(point[0],point[1]-1)
        right=(point[0],point[1]+1)
        up_bound=self.upbound_nearest(point,row_size,col_size)
        down_bound=self.downbound_nearest(point,row_size,col_size)
        right_bound=self.rightbound_nearest(point,row_size,col_size)
        left_bound=self.leftbound_nearest(point,row_size,col_size)

        if left in walls:
            if up_bound==-1 or down_bound==-1:
                return False
            else:
                for m in range(up_bound+1,down_bound):
                    if (m,point[1]-1) not in walls:
                        return False
            return True
        if right in walls:
            if up_bound==-1 or down_bound==-1:
                return False
            else:
                for m in range(up_bound+1,down_bound):
                    if (m,point[1]+1) not in walls:
                        return False
            return True
        if top in walls:
            if right_bound==-1 or left_bound==-1:
                return False
            else:
                for m in range(left_bound+1,right_bound):
                    if (point[0]-1,m) not in walls:
                        return False
            return True
        if down in walls:
            if right_bound==-1 or left_bound==-1:
                return False
            else:
                for m in range(left_bound+1,right_bound):
                    if (point[0]+1,m) not in walls:
                        return False
            return True
        return False
        

    def valid_moves(self):
        valid_moves=set()
        for dir in move_directions:
            a=(self.player[0]+dir[0],self.player[1]+dir[1])
            b=(self.player[0]+2*dir[0],self.player[1]+2*dir[1])
            if a not in walls:
                if a in self.blocks:
                    if b not in self.blocks.union(walls):
                        if b not in deadlocks:
                            valid_moves.add(dir)
                else:
                    valid_moves.add(dir)
        return valid_moves
    
    def move(self,direction):
        direction_tmp=deepcopy(direction)
        direction_letter=""
        if(direction_tmp==(-1,0)):
            direction_letter='u'
        elif direction_tmp==(1,0):
            direction_letter='d'
        elif direction_tmp==(0,-1):
            direction_letter='l'
        else:
            direction_letter='r'  
        player_pos_new=(self.player[0]+direction[0],self.player[1]+direction[1])
        box_pos_new=(self.player[0]+(2*direction[0]),self.player[1]+2*(direction[1]))
        if player_pos_new in self.blocks:
            self.blocks.remove(player_pos_new)
            self.blocks.add(box_pos_new)
            if(direction_letter=='u'):
                direction_letter='U'
            elif(direction_letter=='l'):
                direction_letter='L'
            elif(direction_letter=='r'):
                direction_letter='R'
            else:
                direction_letter='D'
        self.player=player_pos_new
        self.move_history.append(direction_letter)
    def goal_reached(self):
        if all(item in self.blocks for item in goals):
            for item in self.blocks:
                self.block_on_goal.add(item)
            return True
        else:
            return False
    def get_result(self):
        path=''
        for char in self.move_history:
            path+=char
        return path
    

In [6]:
def children(board):
    moves=board.valid_moves()
    children_list=set()
    for move in moves:
        board_copy=deepcopy(board)
        board_copy.move(move)
        var1 = 0
        var2 = 0
        numg = 0
        numb = 0
        reqarr = []
        for i in goals:
          numg +=1
        for j in board_copy.blocks:
          numb+=1
        for yo in range(numg):
          reqarr.append([])
          for ys in range(numb):
            reqarr[yo].append(0)
        # print(numg, numb)
        for goal in goals:
          var2 =0
          for block in board_copy.blocks:
            # print(goal)
            # print(block)
            # print(var1, var2)
            reqarr[var1][var2] = dist_to_goal[goal][block]
            var2+=1
          var1+=1
        board_copy.init_reqarr(reqarr)
        # print(board_copy.reqarr)
        children_list.add(board_copy)
    return children_list
def goal_pull(board,p1,p2):
    return dist_to_goal[p2][p1]
def minmatch(board):
    yep = board.reqarr
    y = np.array([np.array(xi) for xi in yep])
    row_ind, col_ind = linear_sum_assignment(y)
    lesgo = y[row_ind, col_ind].sum()
    return lesgo
def manhattan(p1,p2):
    p1_x=p1[0]
    p1_y=p1[1]
    p2_x=p2[0]
    p2_y=p2[1]
    return abs(p1_x-p2_x)+ abs(p1_y-p2_y)
def euclidian(p1,p2):
    p1_x=p1[0]
    p1_y=p1[1]
    p2_x=p2[0]
    p2_y=p2[1]
    return math.sqrt(float(math.pow(p1_x-p2_x,2))+float(math.pow(p1_y-p2_y,2)))
def h(board,heuristic):

        sum=0.0
        dist_player=999999999
        if heuristic=="minmatch":
            sum=minmatch(board)
            return sum
        elif heuristic=="goalpull":
            sum=0
            for goal in goals:
                for block in board.blocks:
                    sum=sum+board.dist_to_goal[goal][block]
            return sum
        else:
            for block in board.blocks:
                if heuristic=="manhattan":
                    curr_dist_player=manhattan(board.player,block)
                elif heuristic=="euclidian":
                    curr_dist_player=euclidian(board.player,block)
                else:
                    print('error give correct heuristic')
                if curr_dist_player<dist_player:
                    dist_player=curr_dist_player
            sum=sum+float(dist_player)
            for block in board.blocks:
                dist_block=999999999
                for goal in goals:
                    if heuristic=="manhattan":
                        curr_dist_block=manhattan(block,goal)
                    elif heuristic=="euclidian":
                        curr_dist_block=euclidian(block,goal)
                    else:
                        print('error give correct heuristic')
                    if curr_dist_block<dist_block:
                        dist_block=curr_dist_block
                    sum=sum+float(dist_block)
            return sum


def Astar(board: State,heuristic):
    init_time=time.time()
    open = makefheap()
    closed= set()
    visited_set=set()
    board.f=h(board,heuristic)+board.g
    fheappush(open,board)
#     open.put(board)
 
    nodes=0
    while (open.num_nodes):
        curr_state=fheappop(open)
#         print(curr_state.f)
        if curr_state.goal_reached():
            end_time=time.time()
          
            print("Path found is: ",end="")
            print(curr_state.get_result())
            print('Time taken: '+str(end_time-init_time)+' secs')
            print("--------------------------------------------------------")
#             return True
            return curr_state.get_result(), str(end_time-init_time)
        closed.add(curr_state)
        for x in children(curr_state):
            if x in closed:
                        continue
            # if ht.if_present(x):
            #     continue
            x.g=curr_state.g
            x.f=x.g+h(x,heuristic)
#             open.put(x)
            fheappush(open,x)
#             visited_set.insert(0,curr_state)
            visited_set.add(curr_state)
    return False

In [7]:
class Sokoban:
    def __init__(self):
        self.col_size=0
        self.row_size=0
        walls.clear()
        goals.clear()
        empty_tiles.clear()
        deadlocks.clear()
        dist_to_goal.clear()
    def new_game(self,levl):
        move_list=[]
        new_board=State(move_list)
        y=1
        x=1
        for line in levl:
            row=[]
            for char in line:
                if char == '#':
                        walls.add((x,y))
                elif char == '.':
                        goals.add((x,y))
                        empty_tiles.add((x,y))
                elif char == '@':
                        new_board.player=(x,y)
                        empty_tiles.add((x,y))
                elif char == '+':
                        new_board.player=(x,y)
                        goals.add((x,y))
                        empty_tiles.add((x,y))
                elif char == '$':
                        new_board.blocks.add((x,y))
                        empty_tiles.add((x,y))
                elif char == '*':
                        new_board.blocks.add((x,y))
                        goals.add((x,y))
                        empty_tiles.add((x,y))
                elif char == ' ':
                        empty_tiles.add((x,y))
                y+=1
                row.append(char)
            if(self.col_size<y):
                self.col_size=y
            if len(row)!=0:
                x+=1
            y=1
        # print(new_board.blocks)
        # print(new_board.walls)
        self.row_size=x
        new_board.height=self.row_size-1
        new_board.width=self.col_size-1
        # print(self.row_size)
        # print(self.col_size)
        # print(new_board.height)
        # print(new_board.width)
        new_board.add_deadlocks(self.row_size-1,self.col_size-1)
        new_board.goalpull()
        # print("hiii")
        # print(new_board.dist_to_goal)
        var1 = 0
        var2 = 0
        numg = 0
        numb = 0
        reqarr = []
        for i in goals:
          numg +=1
        for j in new_board.blocks:
          numb+=1
        for yo in range(numg):
          reqarr.append([])
          for ys in range(numb):
            reqarr[yo].append(0)
        # print(numg, numb)
        for goal in goals:
          var2 =0
          for block in new_board.blocks:
            # print(goal)
            # print(block)
            # print(var1, var2)
            reqarr[var1][var2] = dist_to_goal[goal][block]
            var2+=1
          var1+=1
        new_board.init_reqarr(reqarr)
        # for i in range(numg):
        #   for j in range(numb):
        #     print(reqarr[i][j])
        return new_board

    def search(self,board,heuristic=None):
        return Astar(board,heuristic)

Run on given test examples

In [ ]:
filename="testExamples.xsb"
with open(filename, 'r') as f:
            reader=f.read()
            lines=reader.split('\n')
# print(lines)
levels=[]
levl=[]
level_number=0
level_false_num=0
for line in lines:
    # print(line)
    if line=='':
        levels.append(levl)
        levl=[]
        level_number=level_number+1   
    elif line[0]==';':
        continue
    else:
        levl.append(line)
# print(levels)
i=1
for levl in levels:
    print('Level: ',end="")
    print(i)
#     print(levl)
    sokoban_game=Sokoban()
    board=sokoban_game.new_game(levl)
    sol_found=sokoban_game.search(board,'minmatch')
    # print(sol_found)
    i+=1
    print()

Level: 1
Path found is: uuRldddrrUdrrruLdlllluuuurrrrdrUUruLLLLLrrrrdddllulldRlulddddrruUUluRRRdLrrUUruLLLLrrrddllllldddRdrUlluurRlldddrruUUllurRRRdLrrUUruLLLrrddlllldldddrrrruLdlUlluurRuldlddrrUUluRRRdLrrUU
Time taken: 2.7844417095184326 secs
--------------------------------------------------------

Level: 2
Path found is: drrrrdrrddllUUruLLLulldRddrRlluuurldRlrRRdrrddllUllluurrrurDllllddrrrUdddlUrrruuLuLLrrdrddlluUruulDDrlDuulLulldRddrRdRUUruulDlLulDrrrDDlddrUUUruLLLrrdrrddLruulldlddrUUUruLL
Time taken: 3.500018358230591 secs
--------------------------------------------------------

Level: 3
Path found is: uulDDDDDurUUllururDrrrrrdddllldddlllluRRluuruUllurRRRRldLLrullDrddldddrrruuurruuruulDDullllulddDDDuurururrrdrddllldddlluRdrUUdlldlluRRRdrUllluuruurrurrdDrdL
Time taken: 15.138929605484009 secs
--------------------------------------------------------

Level: 4
Path found is: dlldddllluurRllddrlrrrurrlrrruuuduuuulllllrlldddRDrrruullDlrurrddlLulluuurrrrrrddddddllllUduLUdrddllluuRlddrr

Wrirting path of each level to the file Results.txt
Make sure that the Results.txt is empty if it is present.
Change the filename below and run the cell.

In [ ]:
filename="testExamples.xsb"
with open(filename, 'r') as f:
            reader=f.read()
            lines=reader.split('\n')
# print(lines)
levels=[]
levl=[]
level_number=0
level_false_num=0
for line in lines:
    # print(line)
    if line=='':
        levels.append(levl)
        levl=[]
        level_number=level_number+1   
    elif line[0]==';':
        continue
    else:
        levl.append(line)
# print(levels)
i=1
for levl in levels:
    print('Level: ',end="")
    line1 = str('Level: ') + str(i)
    print(i)
#     print(levl)
    line2 = levl
    sokoban_game=Sokoban()
    board=sokoban_game.new_game(levl)
    path_found, timet=sokoban_game.search(board,'minmatch')
#     print(sol_found)
    pathh = ""
    for p in path_found:
        pathh = pathh + p
    line22 = "Path found is: " + pathh + "\n"
    line33 = 'Time taken: ' + str(timet)
    
    i+=1
    

    outputFileName = "Results.txt"
    with open(outputFileName, 'a') as filetowrite:
        filetowrite.writelines(pathh + "\n")

Level: 1
Path found is: uuRldddrrUdrrruLdlllluuuurrrrdrUUruLLLLLrrrrdddllulldRlulddddrruUUluRRRdLrrUUruLLLLrrrddllllldddRdrUlluurRlldddrruUUllurRRRdLrrUUruLLLrrddlllldldddrrrruLdlUlluurRuldlddrrUUluRRRdLrrUU
Time taken: 1.4397788047790527 secs
--------------------------------------------------------
Level: 2
Path found is: drrrrdrrddllUUruLLLulldRddrRlluuurldRlrRRdrrddllUllluurrrurDllllddrrrUdddlUrrruuLuLLrrdrddlluUruulDDrlDuulLulldRddrRdRUUruulDlLulDrrrDDlddrUUUruLLLrrdrrddLruulldlddrUUUruLL
Time taken: 1.4758477210998535 secs
--------------------------------------------------------
Level: 3
Path found is: uulDDDDDurUUllururDrrrrrdddllldddlllluRRluuruUllurRRRRldLLrullDrddldddrrruuurruuruulDDullllulddDDDuurururrrdrddllldddlluRdrUUdlldlluRRRdrUllluuruurrurrdDrdL
Time taken: 15.188956022262573 secs
--------------------------------------------------------
Level: 4
Path found is: dlldddllluurRllddrlrrrurrlrrruuuduuuulllllrlldddRDrrruullDlrurrddlLulluuurrrrrrddddddllllUduLUdrddllluuRlddrrru

###**Run** **on** **self** **choosen** **test** **set**:
Our code was able to solve **90** **levels** of **microban** **test** **suite**.

Result of first 50 levels of microban test suite.
The levels are in microban.xsb file

In [9]:
filename="microban.xsb"
with open(filename, 'r') as f:
            reader=f.read()
            lines=reader.split('\n')
# print(lines)
levels=[]
levl=[]
level_number=0
level_false_num=0
for line in lines:
    # print(line)
    if line=='':
        levels.append(levl)
        levl=[]
        level_number=level_number+1   
    elif line[0]==';':
        continue
    else:
        levl.append(line)
# print(levels)
i=1
for levl in levels:
    print('Level: ',end="")
    print(i)
#     print(levl)
    sokoban_game=Sokoban()
    board=sokoban_game.new_game(levl)
    sol_found=sokoban_game.search(board,'minmatch')
    # print(sol_found)
    i+=1
    print()

Level: 1
Path found is: dlUrrrdLullddrUdluuRuulDrddrruLdlUU
Time taken: 0.04461979866027832 secs
--------------------------------------------------------

Level: 2
Path found is: rddLruulDuullddR
Time taken: 0.013204574584960938 secs
--------------------------------------------------------

Level: 3
Path found is: ruuLLLulDllddrrruurrrddlUruLLLddllluurRDrdLuuurDD
Time taken: 0.13112974166870117 secs
--------------------------------------------------------

Level: 4
Path found is: ullDLdRuurrdLLrrddlUruL
Time taken: 0.03558754920959473 secs
--------------------------------------------------------

Level: 5
Path found is: LrldRululDuurrrdLdddllluRuuurrrrdddLUddllURuLdruuulldDrdrdllluR
Time taken: 0.12415385246276855 secs
--------------------------------------------------------

Level: 6
Path found is: ulldllllulllddRluurDRRRRRdrrruullDurrddlLLLullulldlddrUdrUldluuurDRRRRRllllddlUruulldRRRRRdrrrruullDurrddlLLruLLLLulldRRRRRdrrruullDurrddlL
Time taken: 0.17002558708190918 secs
------------

Result of 50-90 levels of microban test suite.
The levels are in microban 50+.xsb file

In [14]:
filename="microban 50+.xsb"
with open(filename, 'r') as f:
            reader=f.read()
            lines=reader.split('\n')
# print(lines)
levels=[]
levl=[]
level_number=0
level_false_num=0
for line in lines:
    # print(line)
    if line=='':
        levels.append(levl)
        levl=[]
        level_number=level_number+1   
    elif line[0]==';':
        continue
    else:
        levl.append(line)
# print(levels)
i=51
for levl in levels:
    print('Level: ',end="")
    print(i)
#     print(levl)
    sokoban_game=Sokoban()
    board=sokoban_game.new_game(levl)
    sol_found=sokoban_game.search(board,'minmatch')
    # print(sol_found)
    i+=1
    print()

Level: 51
Path found is: rddllluURuLrdddrruuLrddllluuurDuldlluurDldRR
Time taken: 0.04686379432678223 secs
--------------------------------------------------------

Level: 52
Path found is: dddLrddlUlluuurRurDlrDulldRllddrUluR
Time taken: 0.04473614692687988 secs
--------------------------------------------------------

Level: 53
Path found is: lUluuRurrDLrrddLUUrddldlluRdrUdullluuRlddrUUdddrurruulLrrddlU
Time taken: 0.21390199661254883 secs
--------------------------------------------------------

Level: 54
Path found is: drURuRRRdrUUddlullldlldrrrUluRRRdrUllllulDudrdrdllluRRdlluluRRRRRRlrRlllluldlldrrrdrUluRRRdrrULdurUruulDulldRDDuuurrdLrdLrldlLLrrruruullldRudurrdLrdldlllLdlllurRRRR
Time taken: 0.9939391613006592 secs
--------------------------------------------------------

Level: 55
Path found is: rdrrrddlUdrddlUUruuLLLulldRRRRurDDDlUddRRluuuLLLulldRRRRurDDDldR
Time taken: 0.1799015998840332 secs
--------------------------------------------------------

Level: 56
Path found is: ulu